## Exploratory Data Analysis

In [ ]:
# Import pandas and matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# Read the restaurants csv file
restaurants = pd.read_csv("../files/paris_restaurants.csv")
print(restaurants)

In [ ]:
# Make a plot of all points
fig, ax = plt.subplots()
ax.plot(restaurants.x, restaurants.y, 'o')
plt.show()

```
import matplotlib.pyplot as plt

# Make a plot of all points
fig, ax = plt.subplots()

# Plot the points with blue color and larger markers
ax.plot(restaurants.x, restaurants.y, 'o', color='blue', markersize=8, label='Restaurants')

# Add grid lines
ax.grid(True)

# Add labels to the axes
ax.set_xlabel('X axis label')
ax.set_ylabel('Y axis label')

# Add a title to the plot
ax.set_title('Plot of Restaurant Locations')

# Add a legend
ax.legend()

# Show the plot
plt.show()
```

### Add a background map

We can visualise the locations of restaurants on a map using the matplotlib and contextily libraries in Python. 
It plots the coordinates of restaurants and overlays them with a basemap for geographical context.

In [ ]:
# Import contextily
import contextily
 
# A figure of all restaurants with background
fig, ax = plt.subplots()
ax.plot(restaurants.x, restaurants.y, 'o', markersize=1)
contextily.add_basemap(ax)
#contextily.add_basemap(ax, source=contextily.providers.CartoDB.Positron)
plt.show()

### Playing with `Geopandas`

In [ ]:
# Import GeoPandas
import geopandas as gpd
 
# Read the Paris districts dataset
arrondissements = gpd.read_file('../files/arrondissements.geojson')

# Inspect the first rows
print(arrondissements.head())
 
# Make a quick visualization of the districts
arrondissements.plot()
plt.show()

In [ ]:
# now sorted
arrondissements = arrondissements.sort_values(by=['c_ar'])
print(arrondissements)

According to the crs, we see that the EPSG Code is 4326.

This means that the XY coordinates are based on longitude and latitude

In [ ]:
print(arrondissements.crs)

To create a projected digial map, we need to covert it to the French (Paris) Coordinate system. The EPSG code is 2154.

In [ ]:
arrondissements = arrondissements.to_crs(epsg=2154)
print(arrondissements)

In [ ]:
# Make a quick visualization of the districts
arrondissements.plot()

for idx, row in arrondissements.iterrows():
    # Get the centroid of the district to place the label
    centroid = row.geometry.centroid
    plt.text(centroid.x, centroid.y, str(row['c_ar']), color='yellow', weight='bold', fontsize=9, ha='center')

plt.show()


As the population data is missing, we are going to download a population data from a spreadsheet and join it to the shapefile 

In [ ]:
districts_pop = pd.read_csv('../files/paris_arrondissements_population.csv')

In [ ]:
districts_pop

The shapefile contains the column `c_ar` as an index for the arrondissements, and `no` for the population. 

Printing the columns of each dfs and their dtypes, we can identify that the data type is identical

In [ ]:
print(arrondissements.columns)
print(districts_pop.columns)

In [ ]:
print(arrondissements['c_ar'].dtypes)
print(districts_pop['no'].dtypes)

In [ ]:
arrondissements_pop = pd.merge(arrondissements,districts_pop, how='left',left_on=['c_ar'],right_on=['no'])

In [ ]:
arrondissements_pop.head()

### Explore the districts of Paris

In [ ]:
# Check what kind of object districts is
print(type(arrondissements_pop))
 
# Check the type of the geometry attribute
print(type(arrondissements_pop.geometry))
 
# Inspect the first rows of the geometry
print(arrondissements_pop.geometry.head())
 


In [ ]:
# Inspect the area of the districts in sq/km2
print(arrondissements_pop.geometry.area/1000000)

### The Magic: transforming csv to Geodataframe

In [ ]:
# Convert it to a GeoDataFrame
restaurants_shp = gpd.GeoDataFrame(restaurants, geometry=gpd.points_from_xy(restaurants.x, restaurants.y))

# create crs
restaurants_shp = restaurants_shp.set_crs('epsg:3857')

# Inspect the first rows of the restaurants GeoDataFrame
print(restaurants_shp.head())
 
# Make a plot of the restaurants
ax = restaurants_shp.plot(markersize=1)


contextily.add_basemap(ax)
plt.show()

In [ ]:
restaurants_paris = restaurants_shp.to_crs(epsg=2154)

## Measuring and Visualising Population Density

In [ ]:
# Add a population density column
arrondissements_pop['population_density'] = arrondissements_pop.Population / arrondissements_pop.area * 10**6
 
# Make a plot of the districts colored by the population density
arrondissements_pop.plot(column='population_density', legend=True)
plt.show()

Name of the districts in Paris

In [ ]:
arrondissements_pop.plot(column='population_density', legend=True)

for idx, row in arrondissements_pop.iterrows():
    # Get the centroid of the district to place the label
    centroid = row.geometry.centroid
    plt.text(centroid.x, centroid.y, str(row['c_ar']), color = 'white', weight='bold',  fontsize=8, ha='center')

plt.show()

## Group by

In [ ]:
# Calculate the number of restaurants of each type
type_counts = restaurants_shp.groupby('type').size()
 
# Print the result
print(type_counts)

### Plotting multiple layers

Another typical pandas functionality is filtering a dataframe: taking a subset of the rows based on a condition (which generates a boolean mask).

In this exercise, we will take the subset of all Asian restaurants, and then make a multi-layered plot. In such a plot, we combine the visualization of several GeoDataFrames on a single figure. To add one layer, we can use the ax keyword of the plot() method of a GeoDataFrame to pass it a matplotlib axes object.

In [ ]:
# Take a subset of the Asian restaurants
asian_restaurants = restaurants_shp[restaurants_shp['type'] == 'Asian restaurant']
asian_restaurants.shape

In [ ]:
# Create a multi-layered plot with specified size
fig, ax = plt.subplots(figsize=(10, 10))  # Adjusted size for clarity

# Plot all restaurants in grey
#restaurants_shp.plot(color='grey', ax=ax)
arrondissements_pop.plot(ax=ax)

# Plot Asian restaurants in red over the existing plot
asian_restaurants.plot(color='red', ax=ax)
#contextily.add_basemap(ax)

# Display the plot
plt.show()

In [ ]:
#points_in_polygon = gpd.sjoin(asian_restaurants, arrondissements, how="inner", predicate='intersects')